# Romance

In [1]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "romance_translated"
method = "pre-translate"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_gpt_response(claim, model=model):
    try:
        user_prompt = prompt_template.format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_gpt_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


/Users/aryan/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Model Output: To determine the accuracy of the claim that "Nigeria spends 7 times less in health care per capita than South Africa," we need to look at the most recent and reliable data on health care expenditure per capita for both countries.

According to the World Bank data for the most recent year available:

- Nigeria's health care expenditure per capita is approximately $74.
- South Africa's health care expenditure per capita is approximately $526.

To compare these figures, we divide South Africa's expenditure by Nigeria's expenditure:

$526 / $74 ≈ 7.11

This calculation shows that South Africa spends a little over 7 times more per capita on health care than Nigeria.

Final Answer: True
Final Answer: True, Actual Answer: False
Wrong response
Model Output: To determine the accuracy of the claim, we need to verify the historical and constitutional context of Senegal. The "blocking quarter" refers to a provision in the Senegalese constitution that required a certain percentage of 

# Indo-Aryan

In [2]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "indo_aryan_translated"
method = "pre-translate"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_gpt_response(claim, model=model):
    try:
        user_prompt = prompt_template.format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_gpt_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: The claim "Photo by George Floyd, who recently died in police torture in Minneapolis, USA" is misleading for several reasons:

1. George Floyd was a victim of police brutality and died during an arrest in Minneapolis, USA, on May 25, 2020. His death was widely reported and led to global protests against police violence and systemic racism.
2. The phrase "police torture" is not the commonly used term to describe the incident. The widely accepted term is "police brutality" or "police violence."
3. George Floyd was not known as a photographer, and there is no widely recognized photo attributed to him.

Given these points, the claim is inaccurate.

Final Answer: False
Final Answer: False, Actual Answer: Mostly false
Wrong response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: False
Correct response
Model Output: Final Answer: False
Final Answer: False, Actual Answer: False
Correct response
Model Output: To determine the accuracy of the claim, we need t

# Kartvelian

In [4]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "kartvelian_translated"
method = "pre-translate"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_gpt_response(claim, model=model):
    try:
        user_prompt = prompt_template.format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_gpt_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: To determine the accuracy of the claim, we need to break down the components and verify the calculations.

1. **Initial Salary Expenses**: The claim states that the salary expenses are more than 1.7 billion. For simplicity, let's assume the salary expenses are exactly 1.7 billion GEL.

2. **Reduction by 20%**: Reducing 1.7 billion by 20% means calculating 20% of 1.7 billion and then subtracting it from the original amount.
   - 20% of 1.7 billion = 0.20 * 1,700,000,000 = 340,000,000 GEL.
   - Reduced salary expenses = 1,700,000,000 - 340,000,000 = 1,360,000,000 GEL.

3. **Savings from Reduction**: The savings from the reduction would be 340,000,000 GEL.

4. **Increasing Pension by 50 GEL**: To understand the impact of increasing pensions by 50 GEL, we need to know the number of pensioners. Let's denote the number of pensioners as \( P \).

5. **Total Cost of Pension Increase**: The total cost of increasing pensions by 50 GEL per pensioner would be \( 50 \times P \).

To v

# Slavic

In [5]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "slavic_translated"
method = "pre-translate"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_gpt_response(claim, model=model):
    try:
        user_prompt = prompt_template.format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_gpt_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: The claim states that "There are different excise dates in other European countries to different quality of petroleum products." This statement is somewhat unclear and ambiguous, but it seems to be suggesting that different European countries have different excise tax rates or dates of implementation for various qualities of petroleum products.

In reality, excise taxes on petroleum products do vary across European countries, and these taxes can be influenced by the quality or type of the petroleum product (e.g., diesel vs. gasoline, high-sulfur vs. low-sulfur fuels). However, the term "excise dates" is not commonly used in this context, and it is unclear what is meant by it. If the intention is to refer to the dates when different excise tax rates are applied or changed, this can also vary by country.

Given the ambiguity of the term "excise dates" but acknowledging the general truth that excise taxes on petroleum products do vary across European countries, the claim can

# Turkic

In [ ]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "turkic_translated"
method = "pre-translate"
model = "gpt-4o-mini"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Define the prompt
prompt_template = "'{claim}' Is this claim 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'? You must always make sure your final response is prefixed with 'Final Answer:' followed by either 'True', 'Mostly True', 'Half True', 'Mostly False', or 'False'."

final_answer_mappings = {
    "true": "true", "mostly true": "mostly true", "half true": "half true",
    "mostly false": "mostly false", "false": "false"
}

def get_gpt_response(claim, model=model):
    try:
        user_prompt = prompt_template.format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output):
    try:
        final_answer = re.search(r'final answer:\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(1).strip()
            return final_answer_mappings.get(response, None)
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    output = get_gpt_response(claim)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")
